# Variational Classifier

In [25]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ

IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [26]:
provider = IBMQ.get_provider(hub='ibm-q', group='open')
device = provider.get_backend('ibmq_ourense')
noise_model = NoiseModel.from_backend(device.properties())

dev = qml.device('qiskit.aer', backend='qasm_simulator', wires=4, noise_model=noise_model)

## Building up the model

### Setting up the layers

In [27]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [28]:
def statePreparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

### Define the quantum circuit

In [29]:
@qml.qnode(dev)
def circuit(weights, x=None):
    statePreparation(x)
    for W in weights:
        layer(W)
        
    return qml.expval(qml.PauliZ(0))

In [30]:
def variational_classifier(var, x=None):
    weights = var[0]
    bias = var[1]
    return circuit(weights, x=x) + bias

### Cost functions

In [31]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += ((l - p)**2)
        
    loss /= len(labels)
    return loss

In [32]:
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss += 1
            
    loss /= len(labels)
    return loss

In [33]:
def cost(var, X, Y):
    predictions = [variational_classifier(var, x=x) for x in X]
    return square_loss(Y, predictions)

## Optimization

In [34]:
data = np.loadtxt('parity.txt')
X = data[:, :-1]
Y = data[:, -1]
Y = Y * 2 - np.ones(len(Y))

for i in range(5):
    print(f'X = {X[i]}, Y = {int(Y[i])}')

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y = 1
X = [0. 0. 1. 0.], Y = 1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y = 1


In [35]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

print(var_init)

(array([[[ 0.01764052,  0.00400157,  0.00978738],
        [ 0.02240893,  0.01867558, -0.00977278],
        [ 0.00950088, -0.00151357, -0.00103219],
        [ 0.00410599,  0.00144044,  0.01454274]],

       [[ 0.00761038,  0.00121675,  0.00443863],
        [ 0.00333674,  0.01494079, -0.00205158],
        [ 0.00313068, -0.00854096, -0.0255299 ],
        [ 0.00653619,  0.00864436, -0.00742165]]]), 0.0)


In [36]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [37]:
var = var_init
for it in range(25):
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    var = opt.step(lambda v: cost(v, X_batch, Y_batch), var)
    
    predictions = [np.sign(variational_classifier(var, x=x)) for x in X]
    acc = accuracy(Y, predictions)
    
    print("Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(it + 1, cost(var, X, Y), acc))

Iter:     1 | Cost: 3.2123858 | Accuracy: 0.5000000 
Iter:     2 | Cost: 1.7413375 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.5585726 | Accuracy: 0.5000000 
Iter:     4 | Cost: 1.1304681 | Accuracy: 0.5000000 
Iter:     5 | Cost: 0.9638177 | Accuracy: 0.6250000 
Iter:     6 | Cost: 1.2072096 | Accuracy: 0.5000000 
Iter:     7 | Cost: 1.0231512 | Accuracy: 0.5000000 
Iter:     8 | Cost: 1.0268359 | Accuracy: 0.5000000 
Iter:     9 | Cost: 1.0293251 | Accuracy: 0.5000000 
Iter:    10 | Cost: 0.9905479 | Accuracy: 0.5625000 
Iter:    11 | Cost: 0.9200014 | Accuracy: 0.6250000 
Iter:    12 | Cost: 0.6923787 | Accuracy: 0.8750000 
Iter:    13 | Cost: 0.6330797 | Accuracy: 0.6875000 
Iter:    14 | Cost: 0.3107374 | Accuracy: 1.0000000 
Iter:    15 | Cost: 0.3100047 | Accuracy: 0.9375000 
Iter:    16 | Cost: 0.1341340 | Accuracy: 1.0000000 
Iter:    17 | Cost: 0.2092206 | Accuracy: 1.0000000 
Iter:    18 | Cost: 0.0594971 | Accuracy: 1.0000000 
Iter:    19 | Cost: 0.0480847 | Accuracy: 1.00